## Deliverable 2. Create a Customer Travel Destinations Map.

In [25]:
# Dependencies and Setup
import pandas as pd
import os
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [9]:
# 1. Import the WeatherPy_database.csv file. 
file_to_load = os.path.join("..","Weather_Database","WeatherPy_Database.csv")
city_data_df = pd.read_csv(file_to_load)
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mangan,IN,27.5167,88.5333,69.57,38,3,5.46,clear sky
1,1,Hermanus,ZA,-34.4187,19.2345,65.10,87,97,15.37,overcast clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,-2.72,97,99,2.42,overcast clouds
3,3,Sitka,US,57.0531,-135.3300,34.52,98,100,2.75,light rain
4,4,Ketchikan,US,55.3422,-131.6461,46.87,93,90,13.80,moderate rain


In [10]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you would like for your trip? '))
max_temp = float(input('What is the maximum temperature you would like for your trip? '))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [11]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] <= max_temp) &
                                       (city_data_df['Max Temp'] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
9,9,Saint-Pierre,RE,-21.3393,55.4781,80.24,83,75,8.05,light rain
14,14,Rikitea,PF,-23.1203,-134.9692,76.37,78,68,16.73,broken clouds
15,15,Busselton,AU,-33.6500,115.3333,83.71,41,70,11.61,broken clouds
16,16,Sao Filipe,CV,14.8961,-24.4956,77.07,71,77,10.80,broken clouds
19,19,Maceio,BR,-9.6658,-35.7353,80.04,83,20,2.30,light rain
30,30,Puerto Colombia,CO,10.9878,-74.9547,80.69,83,45,18.45,scattered clouds
34,34,Nabire,ID,-3.3667,135.4833,82.42,74,100,7.99,overcast clouds
39,39,Makakilo City,US,21.3469,-158.0858,79.12,61,90,4.61,overcast clouds
43,43,Bengkulu,ID,-3.8004,102.2655,82.15,71,100,3.83,overcast clouds
45,45,Hithadhoo,MV,-0.6000,73.0833,81.88,71,99,17.40,overcast clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                145
City                   145
Country                145
Lat                    145
Lng                    145
Max Temp               145
Humidity               145
Cloudiness             145
Wind Speed             145
Current Description    145
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

In [27]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Saint-Pierre,RE,80.24,light rain,-21.3393,55.4781,
14,Rikitea,PF,76.37,broken clouds,-23.1203,-134.9692,
15,Busselton,AU,83.71,broken clouds,-33.6500,115.3333,
16,Sao Filipe,CV,77.07,broken clouds,14.8961,-24.4956,
19,Maceio,BR,80.04,light rain,-9.6658,-35.7353,
30,Puerto Colombia,CO,80.69,scattered clouds,10.9878,-74.9547,
34,Nabire,ID,82.42,overcast clouds,-3.3667,135.4833,
39,Makakilo City,US,79.12,overcast clouds,21.3469,-158.0858,
43,Bengkulu,ID,82.15,overcast clouds,-3.8004,102.2655,
45,Hithadhoo,MV,81.88,overcast clouds,-0.6000,73.0833,


In [28]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row['Lat']
    lng = row['Lng']
    
    # 6c. Get latitude and longitude from DataFrame
    params['location'] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except(IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,Saint-Pierre,RE,80.24,light rain,-21.3393,55.4781,Lindsey Hôtel
14,Rikitea,PF,76.37,broken clouds,-23.1203,-134.9692,People ThankYou
15,Busselton,AU,83.71,broken clouds,-33.6500,115.3333,Observatory Guest House
16,Sao Filipe,CV,77.07,broken clouds,14.8961,-24.4956,Hotel Xaguate
19,Maceio,BR,80.04,light rain,-9.6658,-35.7353,Poutur Pousada
30,Puerto Colombia,CO,80.69,scattered clouds,10.9878,-74.9547,Hotel El Emigrante
34,Nabire,ID,82.42,overcast clouds,-3.3667,135.4833,Hotel Nusantara Nabire
39,Makakilo City,US,79.12,overcast clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club
43,Bengkulu,ID,82.15,overcast clouds,-3.8004,102.2655,Grage Hotel Bengkulu
45,Hithadhoo,MV,81.88,overcast clouds,-0.6000,73.0833,Scoop Guest House


In [33]:
hotel_df.count()

City                   145
Country                145
Max Temp               145
Current Description    145
Lat                    145
Lng                    145
Hotel Name             145
dtype: int64

In [35]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df

In [39]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [44]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# <dt>Max Temp</dt><dd>{Max Temp} °F</dd>

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [45]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))